<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 2: Practical - LDA & QDA <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Arshia Soltani Moakhar
    
____

### Full Name : Pardis Sadat Zahraie
### Student Number : 99109777
___

In [176]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Exploration (20 points)

In [177]:
!wget https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv

--2023-04-17 17:04:56--  https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv [following]
--2023-04-17 17:04:56--  https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89401 (87K) [text/plain]
Saving to: ‘dataset-train.csv.4’

dataset-train.csv.4 100%[===================>]  87.31K  --.-KB/s    in 0.008s  

2023-04-17 17:04:56 (10.3 MB/s) - ‘dataset-train

Load the dataset as a dataframe

In [178]:
df = pd.read_csv('dataset-train.csv')

Get the minimum "Area Income" for all Italians

In [179]:
italy_df = df[df["Country"] == "Italy"]
min_italy = italy_df["Area Income"].min()
print("The minimum Area Income for all Italians:", min_italy)

The minimum Area Income for all Italians: 54286.1


Make a new dataframe called "sample_df" by sampling 100 rows from the previously made dataframe

In [180]:
sample_df = df.sample(n=100, random_state=1)

Compare all men and women aged less than 30 years in terms of their average "Daily Internet Usage". Your result should contain the average daily usage for each age and gender, seperately.

In [181]:

# Filter the DataFrame to include only men and women aged less than 30 years
men = df[(df["Gender"] == "male") & (df["Age"] < 30)]
women = df[(df["Gender"] == "female") & (df["Age"] < 30)]

avg__men = men["Daily Internet Usage"].mean()
avg__women = women["Daily Internet Usage"].mean()

print("Average daily internet usage for men ages < 30 : {:.2f} hours".format(avg__men))
print("Average daily internet usage for women ages < 30 : {:.2f} hours".format(avg__women))

Average daily internet usage for men ages < 30 : 200.08 hours
Average daily internet usage for women ages < 30 : 193.46 hours


In [182]:
# if we want explicitly for each age:
# Filter the DataFrame to include only rows where age is less than 30
df_filtered = df[df["Age"] < 30]
avg_gender = df_filtered.groupby(["Age", "Gender"])["Daily Internet Usage"].mean()
newavg_gender = avg_gender.reset_index()
print(newavg_gender)

    Age  Gender  Daily Internet Usage
0    19  female            184.940000
1    19    male            189.523333
2    20  female            202.160000
3    20    male            198.982500
4    21  female            186.980000
5    21    male            212.815000
6    22  female            200.070000
7    22    male            188.837500
8    23  female            187.105714
9    23    male            184.615000
10   24  female            178.693333
11   24    male            200.055556
12   25  female            193.618571
13   25    male            206.527857
14   26  female            192.741875
15   26    male            203.916250
16   27  female            189.157692
17   27    male            205.916154
18   28  female            205.873333
19   28    male            196.416842
20   29  female            191.191200
21   29    male            202.005714


# Preprocessing (30 points)

Remove the following columns: "Ad Topic Line", "Timestamp"

In [183]:
df = df.drop(["Ad Topic Line", "Timestamp"], axis=1)
print(df)

     Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                       68.95   35     61833.90                256.09   
1                       80.23   31     68441.85                193.77   
2                       69.47   26     59785.94                236.50   
3                       74.15   29     54806.18                245.89   
4                       68.37   35     73889.99                225.58   
..                        ...  ...          ...                   ...   
795                     75.64   29     69718.19                204.82   
796                     79.22   27     63429.18                198.79   
797                     77.05   34     65756.36                236.08   
798                     66.83   46     77871.75                196.17   
799                     76.20   24     47258.59                228.81   

                 City     Country  Clicked on Ad  Gender  
0         Wrightburgh     Tunisia              0  female  
1    

Convert the "City" and "Country" columns to their one-hot encoded versions.
For each set of one-hot encoded columns, remove one of them to make the columns not correlated. Finally, remove the original columns named "City" and "Country".

In [184]:
# I made a csv file for myself as well to use it latter for better analysis
df = pd.get_dummies(df, columns=["City", "Country"])
df = df.drop(["City_Zacharystad", "Country_Zimbabwe"], axis=1)
file_path = 'no_city_country.csv'
df.to_csv(file_path, index=False)
df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Gender,City_Adamsbury,City_Alanview,City_Alexanderfurt,City_Alexanderview,...,Country_United States of America,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia
0,68.95,35,61833.90,256.09,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,79.22,27,63429.18,198.79,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,77.05,34,65756.36,236.08,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
print(df.columns)

Index(['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Clicked on Ad', 'Gender', 'City_Adamsbury',
       'City_Alanview', 'City_Alexanderfurt', 'City_Alexanderview',
       ...
       'Country_United States of America', 'Country_Uruguay',
       'Country_Uzbekistan', 'Country_Vanuatu', 'Country_Venezuela',
       'Country_Vietnam', 'Country_Wallis and Futuna',
       'Country_Western Sahara', 'Country_Yemen', 'Country_Zambia'],
      dtype='object', length=1017)


In [186]:
# to check if we have done it correct
if 'City' in df.columns:
    print("Wrong")
else:
    print("Correct :)")

Correct :)


In [187]:
# to check if we have done it correct
if 'Country' in df.columns:
    print("Wrong")
else:
    print("Correct :)")

Correct :)


Convert "Gender" column to a new binary column (having only 0 and 1 values). Then remove the original column.

In [188]:
df['Gender_binary'] = df['Gender'].map({'female': 0, 'male': 1})
df.drop('Gender', axis=1, inplace=True)
print(df)

     Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                       68.95   35     61833.90                256.09   
1                       80.23   31     68441.85                193.77   
2                       69.47   26     59785.94                236.50   
3                       74.15   29     54806.18                245.89   
4                       68.37   35     73889.99                225.58   
..                        ...  ...          ...                   ...   
795                     75.64   29     69718.19                204.82   
796                     79.22   27     63429.18                198.79   
797                     77.05   34     65756.36                236.08   
798                     66.83   46     77871.75                196.17   
799                     76.20   24     47258.59                228.81   

     Clicked on Ad  City_Adamsbury  City_Alanview  City_Alexanderfurt  \
0                0               0              0 

In [189]:
gendere_column = df['Gender_binary']
print(gendere_column)

0      0
1      1
2      0
3      1
4      0
      ..
795    1
796    1
797    0
798    1
799    1
Name: Gender_binary, Length: 800, dtype: int64


In [190]:
# to check if we have done it correct
if 'Gender' in df.columns:
    print("Wrong")
else:
    print("Correct :)")

Correct :)


By dropping the "Clicked on Ad" column, make a new dataframe called "features_df"

In [191]:
features_df = df.drop('Clicked on Ad', axis=1)
print(features_df)

     Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                       68.95   35     61833.90                256.09   
1                       80.23   31     68441.85                193.77   
2                       69.47   26     59785.94                236.50   
3                       74.15   29     54806.18                245.89   
4                       68.37   35     73889.99                225.58   
..                        ...  ...          ...                   ...   
795                     75.64   29     69718.19                204.82   
796                     79.22   27     63429.18                198.79   
797                     77.05   34     65756.36                236.08   
798                     66.83   46     77871.75                196.17   
799                     76.20   24     47258.59                228.81   

     City_Adamsbury  City_Alanview  City_Alexanderfurt  City_Alexanderview  \
0                 0              0           

Make a new dataframe called "target_df" containing only the "Clicked on Ad" column

In [192]:
target_df = df[['Clicked on Ad']]
print(target_df)

     Clicked on Ad
0                0
1                0
2                0
3                0
4                0
..             ...
795              0
796              0
797              0
798              0
799              0

[800 rows x 1 columns]


Make two numpy arrays called "x" and "y" from "features_df" and "target_df" dataframes, respectively.

In [193]:
x = features_df.values
y = target_df.values

In [194]:
# again I created them to check if I was doing right
file_path = 'training_feature.csv'
features_df.to_csv(file_path, index=False)
file_path = 'training_target.csv'
target_df.to_csv(file_path, index=False)

# Training Models (40 points)

Split the dataset (`x` and `y`) into "training" and "testing" parts containing 80% and 20% of the data respectively.

In [195]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


Train both LDA and QDA models on the data using `sklearn`

In [196]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train.ravel())

LinearDiscriminantAnalysis()

In [197]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train.ravel())

/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

Train LDA and QDA using your own code!

Linear Discriminant Analysis

In [198]:
# jadid 
shrinkage=0.1
# I implemented this method using regularization. regularization is applied to the covariance matrix to make it full rank. instead of pseudo inverse and the results
# were promising :)

In [199]:
class LinearDiscriminantAnalysis_OWN:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self, shrinkage=1e-5):
        self.means = None
        self.covariance = None
        # the proportion of each class in the entire population or dataset we are sampling from
        # because its classification 
        self.priors = None
        self.classes = None
        #adding a scaled identity matrix to the sample covariance matrix.
        self.shrinkage = shrinkage

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        self.means = np.array([X[y == c].mean(axis=0) for c in self.classes])
        self.covariance = np.zeros((n_features, n_features))
        for c in self.classes:
            class_X = X[y == c]
            self.covariance += np.dot((class_X - self.means[c]).T, (class_X - self.means[c]))
        self.covariance /= n_samples - n_classes
        if self.shrinkage is not None:
            self.covariance += self.shrinkage * np.eye(n_features)
        self.priors = np.array([np.mean(y == c) for c in self.classes])
        return self

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape
        y_pred = []
        for i in range(n_samples):
          #separate the classes that 
          #maximizes the between-class variance and minimizes the within-class variance.
          # using the squared Mahalanobis distance
            discriminants = []
            for j in range(len(self.classes)):
                discriminant = np.dot(np.dot(X[i] - self.means[j], np.linalg.inv(self.covariance)),
                                      (X[i] - self.means[j]).T) + 2 * np.log(self.priors[j])
                discriminants.append(discriminant)
            y_pred.append(self.classes[np.argmin(discriminants)])
        return np.array(y_pred)

In [200]:
# find best shrinkage hyperparameter
# I runned it in my device multiple times and steps and found the less the better I commented it so that it is faster in running the cells
"""
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
param_grid = {'shrinkage': np.arange(0, 1, 0.1)}
lda = LinearDiscriminantAnalysis(solver='lsqr')
grid_search = GridSearchCV(lda, param_grid)
grid_search.fit(x_train, y_train)
best_shrinkage = grid_search.best_params_['shrinkage']
print(best_shrinkage)
"""

"\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.discriminant_analysis import LinearDiscriminantAnalysis\nparam_grid = {'shrinkage': np.arange(0, 1, 0.1)}\nlda = LinearDiscriminantAnalysis(solver='lsqr')\ngrid_search = GridSearchCV(lda, param_grid)\ngrid_search.fit(x_train, y_train)\nbest_shrinkage = grid_search.best_params_['shrinkage']\nprint(best_shrinkage)\n"

Quadratic Discriminant Analysis (Optional) (15 bonus points)

In [201]:
import numpy as np
# kinda the same as LDA_OWN I again used Mahalanobis distance
# this was the first QDA I wrote without regularization and shrinkage just to check how much regularization could make it better
"""
 In LinearDiscriminantAnalysis_OWN, the shared covariance matrix is estimated as the weighted average of the individual covariance matrices,
  and is used to calculate the distance between each sample and the class means.
In QuadraticDiscriminantAnalysis_OWN, the individual covariance matrices are estimated for each class,
  and are used in the prediction step to calculate a Mahalanobis distance for each sample with respect to each class.

"""
class QuadraticDiscriminantAnalysis_OWN:
    """
    Quadratic Discriminant Analysis (QDA) class.
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        self.classes = None
        self.priors_ = None
        self.means_ = None
        self.covariance_matrices_ = None

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        self.priors_ = np.zeros(n_classes)
        self.means_ = np.zeros((n_classes, n_features))
        self.covariance_matrices_ = np.zeros((n_classes, n_features, n_features))
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors_[i] = len(X_c) / n_samples
            self.means_[i] = np.mean(X_c, axis=0)
            self.covariance_matrices_[i] = np.cov(X_c, rowvar=False)
        return self

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape
        n_classes = len(self.classes)
        scores = np.zeros((n_samples, n_classes))

        for i, c in enumerate(self.classes):
            X_mu = X - self.means_[i]
            try:
                inv_cov = np.linalg.inv(self.covariance_matrices_[i])
            except np.linalg.LinAlgError:
                inv_cov = np.linalg.pinv(self.covariance_matrices_[i])
            log_det_cov = np.linalg.slogdet(self.covariance_matrices_[i])[1]
            coef = -0.5 * log_det_cov + np.log(self.priors_[i])
            mahalanobis = np.sum(np.dot(X_mu, inv_cov) * X_mu, axis=1)
            scores[:, i] = coef - 0.5 * mahalanobis
        y_pred = self.classes[np.argmax(scores, axis=1)]
        return y_pred

In [202]:
# Train your own models

In [203]:
#LDA
lda_model_OWN = LinearDiscriminantAnalysis_OWN(shrinkage=0.0000001)
lda_model_OWN.fit(x_train, y_train.ravel())

In [204]:
#QDA
qda_model_OWN = QuadraticDiscriminantAnalysis_OWN()
qda_model_OWN.fit(x_train,  y_train.ravel())

# Evaluation (10 points)

Using all 4 models, predict the labels for all samples belonging to the "testing" part

In [205]:
# y_pred_sklda = 
sklda_predictions=lda.predict(x_test)
# y_pred_skqda = 
skqda_predictions=qda.predict(x_test)
# y_pred_lda = 
ldaown_predictions=lda_model_OWN.predict(x_test)
# y_pred_qda = 
qdaown_predictions = qda_model_OWN.predict(x_test)

Calculate Accuracy, Recall, and F1-Scores

In [206]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
accuracy = accuracy_score(y_test, sklda_predictions)
recall = recall_score(y_test, sklda_predictions, average='weighted')
f1 = f1_score(y_test, sklda_predictions, average='weighted')
print("sklda_predictions")
print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("F1-Score: {:.4f}".format(f1))

sklda_predictions
Accuracy: 0.8625
Recall: 0.8625
F1-Score: 0.8615


In [207]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
accuracy = accuracy_score(y_test, skqda_predictions)
recall = recall_score(y_test, skqda_predictions, average='weighted')
f1 = f1_score(y_test, skqda_predictions, average='weighted')
print("skqda_predictions")
print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("F1-Score: {:.4f}".format(f1))

skqda_predictions
Accuracy: 0.9375
Recall: 0.9375
F1-Score: 0.9375


In [208]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
accuracy = accuracy_score(y_test, ldaown_predictions)
recall = recall_score(y_test, ldaown_predictions, average='weighted')
f1 = f1_score(y_test, ldaown_predictions, average='weighted')
print("ldaown_predictions")
print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("F1-Score: {:.4f}".format(f1))
# my LDA works better must be because of reggularization

ldaown_predictions
Accuracy: 0.9500
Recall: 0.9500
F1-Score: 0.9500


In [209]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
accuracy = accuracy_score(y_test, qdaown_predictions)
recall = recall_score(y_test, qdaown_predictions, average='weighted')
f1 = f1_score(y_test, qdaown_predictions, average='weighted')
print("qdaown_predictions")
print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("F1-Score: {:.4f}".format(f1))
# the bad results are because we used pseudo inverse to solve the problem

qdaown_predictions
Accuracy: 0.4875
Recall: 0.4875
F1-Score: 0.3195


In [210]:
# because it was bad we can redo the calculations using regularization shrinkage

In [211]:
import numpy as np

class QuadraticDiscriminantAnalysis_OWN_REG:
    """
    Quadratic Discriminant Analysis (QDA) class.
    Fits it on data, then uses predict to get results.
    """

    def __init__(self, shrinkage=1e-5):
        self.classes = None
        self.priors_ = None
        self.means_ = None
        self.covariance_matrices_ = None
        self.shrinkage = shrinkage

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        self.priors_ = np.zeros(n_classes)
        self.means_ = np.zeros((n_classes, n_features))
        self.covariance_matrices_ = np.zeros((n_classes, n_features, n_features))
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors_[i] = len(X_c) / n_samples
            self.means_[i] = np.mean(X_c, axis=0)
            self.covariance_matrices_[i] = np.cov(X_c, rowvar=False) + self.shrinkage * np.identity(n_features)
        return self

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape
        n_classes = len(self.classes)
        scores = np.zeros((n_samples, n_classes))
        for i, c in enumerate(self.classes):
            X_mu = X - self.means_[i]
            inv_cov = np.linalg.inv(self.covariance_matrices_[i])
            log_det_cov = np.linalg.slogdet(self.covariance_matrices_[i])[1]
            coef = -0.5 * log_det_cov + np.log(self.priors_[i])
            mahalanobis = np.sum(np.dot(X_mu, inv_cov) * X_mu, axis=1)
            scores[:, i] = coef - 0.5 * mahalanobis
        y_pred = self.classes[np.argmax(scores, axis=1)]
        return y_pred

In [212]:
qda_model_OWN_REG = QuadraticDiscriminantAnalysis_OWN_REG()
qda_model_OWN_REG.fit(x_train,  y_train.ravel())

In [213]:
qdaown_predictions_REG = qda_model_OWN_REG.predict(x_test)

In [214]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
accuracy = accuracy_score(y_test, qdaown_predictions_REG)
recall = recall_score(y_test, qdaown_predictions_REG, average='weighted')
f1 = f1_score(y_test, qdaown_predictions_REG, average='weighted')
print("qdaown_predictions_REG")
print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("F1-Score: {:.4f}".format(f1))

qdaown_predictions_REG
Accuracy: 0.7250
Recall: 0.7250
F1-Score: 0.7240


Compute the ROC AUC Score using `sklearn` (for both LDA and QDA models trained using this library)

In [215]:
lda_probs = lda.predict_proba(x_test)[:, 1]
lda_roc_auc = roc_auc_score(y_test, lda_probs)
print("ROC AUC score for LDA : {:.4f}".format(lda_roc_auc))

ROC AUC score for LDA : 0.9212


In [216]:
qda_probs = qda.predict_proba(x_test)[:, 1]
qda_roc_auc = roc_auc_score(y_test, qda_probs)
print("ROC AUC score for QDA : {:.4f}".format(qda_roc_auc))

ROC AUC score for QDA : 0.9375


# Kaggle Competition

Link: https://www.kaggle.com/t/988b0aa2db2446fd85cff571d69c6386

In [217]:
# i have done it in multiple ways in analysis we find that country and city doesent have any effect on data so I deleted it and for time stamp i did it once as a single feature and once as many other features to see if time really matter
# which I find out didnt matter but was better when each hour and second was split and also did some with my own implementation and once with sklearn and my own had 0.97 prec and final file is one without sklearn and QDA is suprisingly not very good in this case


In [218]:
# with timestamp as a whole feature
df = pd.read_csv('dataset-train.csv')
df = df.drop(["Ad Topic Line", "Timestamp","City","Country"], axis=1)
print(df)
df['Gender_binary'] = df['Gender'].map({'female': 0, 'male': 1})
df.drop('Gender', axis=1, inplace=True)
features_df = df.drop('Clicked on Ad', axis=1)
target_df = df[['Clicked on Ad']]
x = features_df.values
y = target_df.values
file_path = 'training_feature.csv'
num_columns = features_df.shape[1]
features_df.to_csv(file_path, index=False)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
lda = LinearDiscriminantAnalysis_OWN(shrinkage=0.000001)
lda.fit(x_train,  y_train.ravel())
df = pd.read_csv('assignment2-test-data.csv')
df = df.drop(["Ad Topic Line", "Timestamp","City","Country","ID"], axis=1)
print(df)
df['Gender_binary'] = df['Gender'].map({'female': 0, 'male': 1})
df.drop('Gender', axis=1, inplace=True)
x = df.values
num_columns = df.shape[1]
file_path = 'finalxx.csv'
df.to_csv(file_path, index=False)
import pandas
LDA_predictions_REG = lda.predict(x)
id_arr = np.arange(0, 200)
df = pandas.DataFrame({'Clicked on Ad': LDA_predictions_REG, 'ID': id_arr})
df.to_csv('outputssss.csv', index=False)

     Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                       68.95   35     61833.90                256.09   
1                       80.23   31     68441.85                193.77   
2                       69.47   26     59785.94                236.50   
3                       74.15   29     54806.18                245.89   
4                       68.37   35     73889.99                225.58   
..                        ...  ...          ...                   ...   
795                     75.64   29     69718.19                204.82   
796                     79.22   27     63429.18                198.79   
797                     77.05   34     65756.36                236.08   
798                     66.83   46     77871.75                196.17   
799                     76.20   24     47258.59                228.81   

     Clicked on Ad  Gender  
0                0  female  
1                0    male  
2                0  female  
3      

In [219]:
# with timestamp as multiple features
df = pd.read_csv('dataset-train.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Month'] = df['Timestamp'].dt.month
df['Daymonth'] = df['Timestamp'].dt.day
df['Dayweek'] = df['Timestamp'].dt.dayofweek
df['Hour'] = df['Timestamp'].dt.hour  
df = df.drop(["Ad Topic Line","City","Country","Timestamp"], axis=1)
print(df)
df['Gender_binary'] = df['Gender'].map({'female': 0, 'male': 1})
df.drop('Gender', axis=1, inplace=True)
features_df = df.drop('Clicked on Ad', axis=1)
target_df = df[['Clicked on Ad']]
x = features_df.values
y = target_df.values
file_path = 'training_feature.csv'
num_columns = features_df.shape[1]
features_df.to_csv(file_path, index=False)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
lda = LinearDiscriminantAnalysis_OWN(shrinkage=0.00001)
lda.fit(x_train,  y_train.ravel())
df = pd.read_csv('assignment2-test-data.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Month'] = df['Timestamp'].dt.month
df['Daymonth'] = df['Timestamp'].dt.day
df['Dayweek'] = df['Timestamp'].dt.dayofweek
df['Hour'] = df['Timestamp'].dt.hour  
df = df.drop(["Ad Topic Line", "Timestamp","City","Country","ID"], axis=1)
print(df)
df['Gender_binary'] = df['Gender'].map({'female': 0, 'male': 1})
df.drop('Gender', axis=1, inplace=True)
x = df.values
num_columns = df.shape[1]
file_path = 'finalxx.csv'
df.to_csv(file_path, index=False)
import pandas
LDA_predictions_REG = lda.predict(x)
id_arr = np.arange(0, 200)
df = pandas.DataFrame({'Clicked on Ad': LDA_predictions_REG, 'ID': id_arr})
df.to_csv('outputssss5.csv', index=False)


     Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                       68.95   35     61833.90                256.09   
1                       80.23   31     68441.85                193.77   
2                       69.47   26     59785.94                236.50   
3                       74.15   29     54806.18                245.89   
4                       68.37   35     73889.99                225.58   
..                        ...  ...          ...                   ...   
795                     75.64   29     69718.19                204.82   
796                     79.22   27     63429.18                198.79   
797                     77.05   34     65756.36                236.08   
798                     66.83   46     77871.75                196.17   
799                     76.20   24     47258.59                228.81   

     Clicked on Ad  Gender  Month  Daymonth  Dayweek  Hour  
0                0  female      3        27        6     0  
1

In [220]:
# to check if there were any improvements by changing the ways
import pandas as pd
df1 = pd.read_csv('outputssss5.csv')
df2 = pd.read_csv('outputssss.csv')
if df1.equals(df2):
    print('The two CSV files are the same.')
else:
    print('The two CSV files are different.')
    differences = df1.compare(df2)
    print('different:')
    print(differences)

The two CSV files are different.
different:
    Clicked on Ad      
             self other
169           1.0   0.0
